# 重回帰モデル
- 説明変数：condition,type,manufacturer
- ※conditionはラベルエンコーディング
- ※type,manufacturerはワンホットエンコーディング

In [119]:
#ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [120]:
#ドライブのマウント
dir_path = '/content/drive/MyDrive/DataScience/SIGNATE/中古車の価格予測 Student Cup 2023/データ/'
df_train = pd.read_csv(dir_path + 'train.csv')
df_test = pd.read_csv(dir_path + 'test.csv')

# データの読み込み

In [121]:
df_train

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
0,0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,rwd,mid-size,convertible,orange,NaN,27587
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,full-size,sedan,silver,pa,4724
2,2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,full-size,SUV,silver,ks,10931
3,3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,fwd,mid-size,SUV,blue,ny,16553
4,4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,fwd,mid-size,sedan,red,ca,5158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,williamsport,2008,ford,good,6 cylinders,gas,26660,clean,automatic,rwd,compact,truck,black,pa,32212
27528,27528,tulsa,2007,ford,excellent,8 cylinders,gas,108072,clean,automatic,rwd,full-size,pickup,black,NaN,5400
27529,27529,rochester,2019,jeep,like new,6 cylinders,gas,139908,clean,automatic,4wd,mid-size,SUV,white,ny,22227
27530,27530,rochester,2007,jeep,excellent,6 cylinders,gas,112326,clean,automatic,4wd,mid-size,sedan,white,ny,3054


In [21]:
df_test

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,27532,western slope,2015,chevrolet,excellent,4 cylinders,gas,92553,clean,automatic,fwd,full-size,SUV,red,NaN
1,27533,roseburg,2013,nissan,like new,4 cylinders,gas,134385,salvage,automatic,fwd,mid-size,sedan,black,or
2,27534,akron / canton,2011,volkswagen,good,4 cylinders,gas,102489,clean,automatic,fwd,full-size,sedan,black,oh
3,27535,denver,2016,jeep,excellent,6 cylinders,diesel,64310,clean,automatic,4wd,mid-size,SUV,red,co
4,27536,hickory / lenoir,1999,honda,excellent,8 cylinders,gas,180839,rebuilt,automatic,4wd,mid-size,SUV,silver,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,55064,great falls,2016,gmc,excellent,6 cylinders,gas,90902,rebuilt,automatic,fwd,full-size,truck,black,mt
27533,55065,las vegas,2012,nissan,excellent,4 cylinders,gas,27234,rebuilt,automatic,fwd,mid-size,sedan,white,nv
27534,55066,phoenix,2002,bmw,excellent,6 cylinders,gas,99761,clean,automatic,rwd,full-size,coupe,blue,az
27535,55067,colorado springs,2006,bmw,excellent,6 cylinders,gas,162279,clean,automatic,4wd,mid-size,sedan,silver,co


# 欠損値処理＆データクレンジング

In [22]:
df_train.isnull().sum()

id                 0
region             0
year               0
manufacturer       0
condition          0
cylinders          0
fuel            1239
odometer           0
title_status     456
transmission       0
drive              0
size               0
type             456
paint_color        0
state           3304
price              0
dtype: int64

In [44]:
df_train_ = df_train.dropna()
df_train_

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,full-size,sedan,silver,pa,4724
2,2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,full-size,SUV,silver,ks,10931
3,3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,fwd,mid-size,SUV,blue,ny,16553
4,4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,fwd,mid-size,sedan,red,ca,5158
5,5,florence / muscle shoals,2013,nissan,good,4 cylinders,gas,133208,clean,automatic,4wd,full-size,SUV,black,al,7941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27526,27526,ventura county,2015,volkswagen,like new,4 cylinders,gas,61568,rebuilt,automatic,fwd,compact,sedan,black,ca,6602
27527,27527,williamsport,2008,ford,good,6 cylinders,gas,26660,clean,automatic,rwd,compact,truck,black,pa,32212
27529,27529,rochester,2019,jeep,like new,6 cylinders,gas,139908,clean,automatic,4wd,mid-size,SUV,white,ny,22227
27530,27530,rochester,2007,jeep,excellent,6 cylinders,gas,112326,clean,automatic,4wd,mid-size,sedan,white,ny,3054


In [45]:
df_train_.isnull().sum()

id              0
region          0
year            0
manufacturer    0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
size            0
type            0
paint_color     0
state           0
price           0
dtype: int64

In [46]:
df_train_['cylinders'] = df_train_['cylinders'].replace({'3 cylinders':3,'4 cylinders':4,'5 cylinders':5,'6 cylinders':6,'8 cylinders':8,'10 cylinders':10,'12 cylinders':12,'other':0})
df_train_['cylinders'].unique()

<ipython-input-46-888d5362ca93>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_['cylinders'] = df_train_['cylinders'].replace({'3 cylinders':3,'4 cylinders':4,'5 cylinders':5,'6 cylinders':6,'8 cylinders':8,'10 cylinders':10,'12 cylinders':12,'other':0})


array([ 8,  6,  4,  0, 10, 12,  3,  5])

In [47]:
df_train_['year'] = df_train['year'].replace({3015:2015, 2999:1999, 3008:2008, 3017:2017})
df_train_['year'].unique()

<ipython-input-47-74f8fc43c4c4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_['year'] = df_train['year'].replace({3015:2015, 2999:1999, 3008:2008, 3017:2017})


array([2013, 1998, 2014, 2005, 2006, 2012, 2010, 2018, 1999, 1996, 2008,
       2015, 2007, 2016, 2003, 1992, 2004, 2009, 2001, 2011, 1988, 1971,
       1994, 2002, 1989, 1995, 1997, 1990, 1983, 2017, 2019, 1987, 1953,
       1978, 1984, 1973, 1980, 1993, 1968, 2000, 2020, 1967, 1976, 1985,
       1982, 1949, 1986, 1960, 1991, 1957, 1962, 1963, 1948, 1974, 2021,
       1970, 1977, 1956, 1961, 1939, 1972, 1969, 1931, 1979, 1981, 1966,
       1942, 1940, 1965, 1955, 1975, 1936, 1959, 1952, 1950, 1964, 1921,
       1947, 1951, 1937, 1922, 1946, 1935, 1928, 2022, 1938, 1941, 1943,
       1954, 1932, 1958, 1929, 1934, 1944, 1925, 1930])

In [48]:
df_train_['manufacturer'].unique()

array(['toyota', 'ford', 'nissan', 'volkswagen', 'bmw', 'FORD', 'buick',
       'subaru', 'ram', 'TOYOTA', 'dodge', 'ｂｍｗ', 'chevrolet', 'RAM',
       'BMW', 'jeep', 'honda', 'NISSAN', 'hyundai', 'chrysler', 'gmc',
       'mazda', 'audi', 'lincoln', 'kia', 'lexus', 'mercedes-benz',
       'mitsubishi', 'CHEVROLET', 'GMC', 'BUICK', 'infiniti', 'MINI',
       'mercury', 'cadillac', 'pontiac', 'acura', 'HONDA', 'ACURA', 'KIA',
       'fiat', 'jaguar', 'ｒａｍ', 'mini', 'LEXUS', 'ｆｏｒｄ', 'VOLKSWAGEN',
       'ｃｈｅｖｒｏｌｅｔ', 'ｋｉａ', 'SUBARU', 'ｎｉｓｓａｎ', 'DODGE', 'volvo', 'MAZDA',
       'ｈｏｎｄａ', 'ｍａｚｄａ', 'INFINITI', 'CADILLAC', 'JEEP', 'HYUNDAI',
       'saturn', 'ｔｏｙｏｔａ', 'MERCEDES-BENZ', 'AUDI', 'ＶＯＬＫＳＷＡＧＥＮ',
       'MERCURY', 'ROVER', 'ｄｏｄｇｅ', 'porsche', 'ｖｏｌｖｏ', 'JAGUAR',
       'SATURN', 'ｊｅｅｐ', 'rover', 'ｓｕｂａｒｕ', 'CHRYSLER', 'PONTIAC',
       'ｍｉｔｓｕｂｉｓｈｉ', 'LINCOLN', 'ＴＯＹＯＴＡ', 'ｃｈｒｙｓｌｅｒ', 'nisѕan',
       'MITSUBISHI', 'ｂｕｉｃｋ', 'ｇｍｃ', 'ｖｏｌｋｓｗａｇｅｎ', 'ＦＯＲＤ', 'ＨＹＵＮＤＡＩ',
       'ＭＡＺＤＡ', 'land rover

In [49]:
df_train_['manufacturer'] = df_train_['manufacturer'].str.normalize('NFKC')
df_train_['manufacturer'] = df_train_['manufacturer'].str.lower()
df_train_['manufacturer'].unique()

<ipython-input-49-5194cd85f680>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_['manufacturer'] = df_train_['manufacturer'].str.normalize('NFKC')
<ipython-input-49-5194cd85f680>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_['manufacturer'] = df_train_['manufacturer'].str.lower()


array(['toyota', 'ford', 'nissan', 'volkswagen', 'bmw', 'buick', 'subaru',
       'ram', 'dodge', 'chevrolet', 'jeep', 'honda', 'hyundai',
       'chrysler', 'gmc', 'mazda', 'audi', 'lincoln', 'kia', 'lexus',
       'mercedes-benz', 'mitsubishi', 'infiniti', 'mini', 'mercury',
       'cadillac', 'pontiac', 'acura', 'fiat', 'jaguar', 'volvo',
       'saturn', 'rover', 'porsche', 'nisѕan', 'land rover',
       'aston-martin', 'subαru', 'alfa-romeo', 'toyotа', 'sαturn',
       'niѕsan', 'аcura', 'vоlkswagen', 'lexuѕ', 'ᴄhrysler'], dtype=object)

In [50]:
df_train_['manufacturer'] = df_train_['manufacturer'].replace({'niѕsan':'nissan','nisѕan':'nissan','subαru':'subaru','toyotа':'toyota','sαturn':'saturn','аcura':'acura','ᴄhrysler':'chrysler','vоlkswagen':'volkswagen','lexuѕ':'lexus'})
df_train_['manufacturer'].unique()

<ipython-input-50-5d6aecf04c28>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_['manufacturer'] = df_train_['manufacturer'].replace({'niѕsan':'nissan','nisѕan':'nissan','subαru':'subaru','toyotа':'toyota','sαturn':'saturn','аcura':'acura','ᴄhrysler':'chrysler','vоlkswagen':'volkswagen','lexuѕ':'lexus'})


array(['toyota', 'ford', 'nissan', 'volkswagen', 'bmw', 'buick', 'subaru',
       'ram', 'dodge', 'chevrolet', 'jeep', 'honda', 'hyundai',
       'chrysler', 'gmc', 'mazda', 'audi', 'lincoln', 'kia', 'lexus',
       'mercedes-benz', 'mitsubishi', 'infiniti', 'mini', 'mercury',
       'cadillac', 'pontiac', 'acura', 'fiat', 'jaguar', 'volvo',
       'saturn', 'rover', 'porsche', 'land rover', 'aston-martin',
       'alfa-romeo'], dtype=object)

In [59]:
df_train_['size'] = df_train_['size'].replace({'full−size':'full-size','fullーsize':'full-size','subーcompact':'sub-compact','midーsize':'mid-size','mid−size':'mid-size'})
df_train_['size'].unique()

<ipython-input-59-63b4ffecfcf7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_['size'] = df_train_['size'].replace({'full−size':'full-size','fullーsize':'full-size','subーcompact':'sub-compact','midーsize':'mid-size','mid−size':'mid-size'})


array(['sub-compact', 'mid-size', 'compact', 'full-size'], dtype=object)

In [60]:
df_train_

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
1,1,state college,2013,toyota,fair,8,gas,172038,clean,automatic,rwd,sub-compact,sedan,silver,pa,4724
2,2,wichita,1998,ford,good,6,gas,152492,clean,automatic,fwd,sub-compact,SUV,silver,ks,10931
3,3,albany,2014,ford,excellent,4,gas,104118,clean,manual,fwd,mid-size,SUV,blue,ny,16553
4,4,redding,2005,ford,excellent,6,gas,144554,clean,manual,fwd,mid-size,sedan,red,ca,5158
5,5,florence / muscle shoals,2013,nissan,good,4,gas,133208,clean,automatic,4wd,sub-compact,SUV,black,al,7941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27526,27526,ventura county,2015,volkswagen,like new,4,gas,61568,rebuilt,automatic,fwd,compact,sedan,black,ca,6602
27527,27527,williamsport,2008,ford,good,6,gas,26660,clean,automatic,rwd,compact,truck,black,pa,32212
27529,27529,rochester,2019,jeep,like new,6,gas,139908,clean,automatic,4wd,mid-size,SUV,white,ny,22227
27530,27530,rochester,2007,jeep,excellent,6,gas,112326,clean,automatic,4wd,mid-size,sedan,white,ny,3054


In [64]:
df_train_['condition'].unique()

array(['fair', 'good', 'excellent', 'like new', 'salvage', 'new'],
      dtype=object)

In [65]:
condition = df_train_.groupby('condition')['price'].mean()
condition = condition.sort_values(ascending=False)
condition

condition
like new     16399.813872
new          15621.181818
excellent    14613.243987
good         11077.342206
fair          7255.832045
salvage       6989.758621
Name: price, dtype: float64

In [66]:
df_train_['condition'] = df_train_['condition'].replace({'salvage':0,'fair':1,'good':2,'excellent':3,'like new':4,'new':5})
df_train_['condition'].unique()

<ipython-input-66-02c61553ad89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_['condition'] = df_train_['condition'].replace({'salvage':0,'fair':1,'good':2,'excellent':3,'like new':4,'new':5})


array([1, 2, 3, 4, 0, 5])

In [68]:
df_train_['type'].unique()

array(['sedan', 'SUV', 'coupe', 'mini-van', 'pickup', 'truck', 'van',
       'hatchback', 'wagon', 'convertible', 'offroad', 'other', 'bus'],
      dtype=object)

In [73]:
feature = df_train[['condition','type','manufacturer','price']]
feature

,condition,type,manufacturer,price
0,excellent,convertible,bmw,27587
1,fair,sedan,toyota,4724
2,good,SUV,ford,10931
3,excellent,SUV,ford,16553
4,excellent,sedan,ford,5158
...,...,...,...,...
27527,good,truck,ford,32212
27528,excellent,pickup,ford,5400
27529,like new,SUV,jeep,22227
27530,excellent,sedan,jeep,3054


In [74]:
feature.isnull().sum()

condition         0
type            456
manufacturer      0
price             0
dtype: int64

In [81]:
feature_ = feature.dropna()
feature_['manufacturer'] = feature_['manufacturer'].str.normalize('NFKC')
feature_['manufacturer'] = feature_['manufacturer'].str.lower()
feature_['manufacturer'] = feature_['manufacturer'].replace({'niѕsan':'nissan','nisѕan':'nissan','subαru':'subaru','toyotа':'toyota','sαturn':'saturn','аcura':'acura','ᴄhrysler':'chrysler','vоlkswagen':'volkswagen','lexuѕ':'lexus'})
feature_['condition'] = feature_['condition'].replace({'salvage':0,'fair':1,'good':2,'excellent':3,'like new':4,'new':5})
feature_

<ipython-input-81-e4eb33e0e9cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_['manufacturer'] = feature_['manufacturer'].str.normalize('NFKC')
<ipython-input-81-e4eb33e0e9cf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_['manufacturer'] = feature_['manufacturer'].str.lower()
<ipython-input-81-e4eb33e0e9cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

,condition,type,manufacturer,price
0,3,convertible,bmw,27587
1,1,sedan,toyota,4724
2,2,SUV,ford,10931
3,3,SUV,ford,16553
4,3,sedan,ford,5158
...,...,...,...,...
27527,2,truck,ford,32212
27528,3,pickup,ford,5400
27529,4,SUV,jeep,22227
27530,3,sedan,jeep,3054


In [77]:
feature_.isnull().sum()

condition       0
type            0
manufacturer    0
price           0
dtype: int64

In [82]:
feature_ = pd.get_dummies(feature_,drop_first=True)
feature_

,condition,price,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,...,manufacturer_nissan,manufacturer_pontiac,manufacturer_porsche,manufacturer_ram,manufacturer_rover,manufacturer_saturn,manufacturer_subaru,manufacturer_toyota,manufacturer_volkswagen,manufacturer_volvo
0,3,27587,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,4724,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2,10931,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,16553,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,5158,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,2,32212,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27528,3,5400,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
27529,4,22227,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27530,3,3054,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 学習用データの準備

In [84]:
x = feature_.drop('price',axis=1)
x

,condition,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,type_sedan,...,manufacturer_nissan,manufacturer_pontiac,manufacturer_porsche,manufacturer_ram,manufacturer_rover,manufacturer_saturn,manufacturer_subaru,manufacturer_toyota,manufacturer_volkswagen,manufacturer_volvo
0,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27528,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
27529,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27530,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [85]:
t = feature_['price']
t

0        27587
1         4724
2        10931
3        16553
4         5158
         ...  
27527    32212
27528     5400
27529    22227
27530     3054
27531    20801
Name: price, Length: 27076, dtype: int64

In [86]:
from sklearn.model_selection import train_test_split
x_train,x_test,t_train,t_test = train_test_split(x,t,test_size=0.3,random_state=0)

# 重回帰モデルの学習

In [87]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [88]:
model.fit(x_train,t_train)

LinearRegression()

In [90]:
print('train score:',model.score(x_train,t_train))
print('test score:',model.score(x_test,t_test))

train score: 0.230308213399529
test score: 0.22260683840333995


# テストデータの準備

In [96]:
df_test_ = df_test[['condition','type','manufacturer']]
df_test_['manufacturer'] = df_test_['manufacturer'].str.normalize('NFKC')
df_test_['manufacturer'] = df_test_['manufacturer'].str.lower()
df_test_['manufacturer'] = df_test_['manufacturer'].replace({'niѕsan':'nissan','nisѕan':'nissan','subαru':'subaru','toyotа':'toyota','sαturn':'saturn','аcura':'acura','ᴄhrysler':'chrysler','vоlkswagen':'volkswagen','lexuѕ':'lexus'})
df_test_['condition'] = df_test_['condition'].replace({'salvage':0,'fair':1,'good':2,'excellent':3,'like new':4,'new':5})
df_test_ = pd.get_dummies(df_test_,drop_first=True)
df_test_

<ipython-input-96-ca5657301cfb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_['manufacturer'] = df_test_['manufacturer'].str.normalize('NFKC')
<ipython-input-96-ca5657301cfb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_['manufacturer'] = df_test_['manufacturer'].str.lower()
<ipython-input-96-ca5657301cfb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

,condition,type_bus,type_convertible,type_coupe,type_hatchback,type_mini-van,type_offroad,type_other,type_pickup,type_sedan,...,manufacturer_nissan,manufacturer_pontiac,manufacturer_porsche,manufacturer_ram,manufacturer_rover,manufacturer_saturn,manufacturer_subaru,manufacturer_toyota,manufacturer_volkswagen,manufacturer_volvo
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27533,3,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
27534,3,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27535,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


# データの予測

In [107]:
y = model.predict(x_test)
y

array([19158.05772907, 26433.76747917, 15741.0887144 , ...,
        8655.65390705, 15329.12452049,  8094.59504485])

In [113]:
df_test['price'] = model.predict(df_test_)
df_test['price']

0        16056.496192
1        12537.070609
2         4362.555785
3        15690.675302
4        12163.376302
             ...     
27532    19670.036786
27533     8994.353206
27534    14324.535081
27535    12198.371311
27536    15690.675302
Name: price, Length: 27537, dtype: float64

In [102]:
df_test

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,27532,western slope,2015,chevrolet,excellent,4 cylinders,gas,92553,clean,automatic,fwd,full-size,SUV,red,NaN
1,27533,roseburg,2013,nissan,like new,4 cylinders,gas,134385,salvage,automatic,fwd,mid-size,sedan,black,or
2,27534,akron / canton,2011,volkswagen,good,4 cylinders,gas,102489,clean,automatic,fwd,full-size,sedan,black,oh
3,27535,denver,2016,jeep,excellent,6 cylinders,diesel,64310,clean,automatic,4wd,mid-size,SUV,red,co
4,27536,hickory / lenoir,1999,honda,excellent,8 cylinders,gas,180839,rebuilt,automatic,4wd,mid-size,SUV,silver,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27532,55064,great falls,2016,gmc,excellent,6 cylinders,gas,90902,rebuilt,automatic,fwd,full-size,truck,black,mt
27533,55065,las vegas,2012,nissan,excellent,4 cylinders,gas,27234,rebuilt,automatic,fwd,mid-size,sedan,white,nv
27534,55066,phoenix,2002,bmw,excellent,6 cylinders,gas,99761,clean,automatic,rwd,full-size,coupe,blue,az
27535,55067,colorado springs,2006,bmw,excellent,6 cylinders,gas,162279,clean,automatic,4wd,mid-size,sedan,silver,co


# 提出ファイルの作成

In [116]:
submission = pd.DataFrame(df_test[['id','price']])
submission

,id,price
0,27532,16056.496192
1,27533,12537.070609
2,27534,4362.555785
3,27535,15690.675302
4,27536,12163.376302
...,...,...
27532,55064,19670.036786
27533,55065,8994.353206
27534,55066,14324.535081
27535,55067,12198.371311


In [118]:
submission.to_csv('submission1.csv',index=False,header=False)